### Importing Libraries

In [8]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Loading and Preprocessing Data

In [9]:
data = load_breast_cancer()
X = data.data
y = data.target  # 0 = malignant, 1 = benign

# Add bias term manually (we'll prepend a column of 1s)
X = np.hstack((np.ones((X.shape[0], 1)), X))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train[:, 1:] = scaler.fit_transform(X_train[:, 1:])
X_test[:, 1:] = scaler.transform(X_test[:, 1:])

### Sigmoid Function

In [10]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

### Logistic Regression with L2 Regularization

In [11]:
def compute_cost(X, y, weights, lambda_reg):
    m = X.shape[0]
    h = sigmoid(np.dot(X, weights))
    eps = 1e-15
    cost = - (1/m) * (np.dot(y, np.log(h + eps)) + np.dot((1 - y), np.log(1 - h + eps)))
    reg_term = (lambda_reg / (2 * m)) * np.sum(weights[1:] ** 2)
    return cost + reg_term

### Logistic Regression

In [12]:
def logistic_regression(X, y, eta, num_iterations, lambda_reg):
    num_samples, num_features = X.shape
    weights = np.zeros(num_features)

    for _ in range(num_iterations):
        predictions = sigmoid(np.dot(X, weights))
        error = predictions - y
        gradient = (1 / num_samples) * (np.dot(X.T, error) + lambda_reg * np.r_[0, weights[1:]])
        weights -= eta * gradient

    return weights

### Training the Model

In [13]:
learning_rate = 0.01
num_iterations = 1000
lambda_reg = 1.0

weights = logistic_regression(X_train, y_train, learning_rate, num_iterations, lambda_reg)

### Predictions and Evaluations

In [14]:
def predict(X, weights, threshold=0.5):
    probs = sigmoid(np.dot(X, weights))
    return (probs >= threshold).astype(int)

y_pred_train = predict(X_train, weights)
y_pred_test = predict(X_test, weights)

train_accuracy = np.mean(y_pred_train == y_train)
test_accuracy = np.mean(y_pred_test == y_test)

print("Training Accuracy:", f"{train_accuracy * 100:.2f}%")
print("Test Accuracy    :", f"{test_accuracy * 100:.2f}%")

train_errors = np.sum(y_pred_train != y_train)
test_errors = np.sum(y_pred_test != y_test)

print("Training Errors  :", train_errors, "out of", len(y_train))
print("Test Errors      :", test_errors, "out of", len(y_test))

Training Accuracy: 98.24%
Test Accuracy    : 98.25%
Training Errors  : 8 out of 455
Test Errors      : 2 out of 114
